In [1]:
import pandas as pd

In [2]:
data_dir='C://Users//mrasskazov//Desktop//DS//Data//ml-latest-small'
tags=pd.read_csv(data_dir+'/tags.csv')
ratings=pd.read_csv(data_dir+'/ratings.csv')
movies=pd.read_csv(data_dir+'/movies.csv')

In [3]:
tags['movie_id']=tags['movieId'].astype('category').cat.codes.copy()

In [4]:
movies_tags = tags[["movieId", "movie_id", 'tag']].drop_duplicates().join(
    movies,
    on="movieId",
    rsuffix="codes",
    lsuffix="movies",
    sort=True
).fillna("None")[["movie_id", "genres",'tag']]

Сливаем все теги для каждого фильма в одну строчку

In [5]:
tags_comb=tags.groupby('movieId')['tag'].apply('|'.join).reset_index()

In [31]:
tags_comb.head()

,movieId,tag
0,1,Pixar
1,5,steve martin
2,47,biblical|crime|dark|disturbing|greed|horror|se...
3,50,thriller
4,104,adam sandler


Создаем матрицу тэгов

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
tags_features=CountVectorizer().fit_transform(tags_comb['tag'])

Добавляем movie_id

In [9]:
from scipy.sparse import hstack, identity

features = hstack([
    identity(tags_comb.shape[0]),
    tags_features
])

Получаем матрицу схожести между фильмами по тэгам

In [10]:
from implicit.als import AlternatingLeastSquares
model=AlternatingLeastSquares()
model.fit(features)

In [25]:
model.similar_items(77)

[(77, 1.0000000000000004),
 (414, 0.53639635169909061),
 (121, 0.47042177617580555),
 (125, 0.41485973638043527),
 (179, 0.35404511840220904),
 (343, 0.32200635119673759),
 (509, 0.30337232060475655),
 (249, 0.26914183245156553),
 (71, 0.22309911718292172),
 (195, 0.21101948061061529)]

In [38]:
list(zip(tags.tag[tags['movie_id']==77],tags.tag[tags['movie_id']==414]))

[('cult film', 'notable nudity'), ('jack nicholson', 'Nudity (Full Frontal)')]

In [51]:
movies_tags

,movie_id,genres,tag
726,0,Adventure|Children|Fantasy,Pixar
592,1,Action|Crime|Thriller,steve martin
339,2,Drama|Romance,biblical
340,2,Drama|Romance,crime
341,2,Drama|Romance,dark
342,2,Drama|Romance,disturbing
343,2,Drama|Romance,greed
344,2,Drama|Romance,horror
345,2,Drama|Romance,serial killer
346,2,Drama|Romance,violent


In [57]:
tags[tags['movie_id']==414]

,userId,movieId,tag,timestamp,movie_id
866,546,53318,notable nudity,1301196436,414
867,546,53318,Nudity (Full Frontal),1301196425,414


In [58]:
movies[movies['movieId']==53318]

,movieId,title,genres
6713,53318,Cashback (2006),Comedy|Drama|Romance
